In [2]:
from genetic_utils import *

# mnist 06

In [ ]:
data, lables = csv_data_loader("mnist-06")

In [ ]:
data.shape

In [ ]:
lables

## VAE

In [ ]:

def train_VAE(data,
              latent_dim = 2,
              hidden_layer_n = [20,18,16],
              num_dims = 10,
              kl_loss_factor = 0.01,
              epochs = 100,
              batch_size = 128
              ):


    """
    Training the VAE on the data
    """

    class Sampling(layers.Layer):
        """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    latent_dim = latent_dim

    encoder_inputs = keras.Input(shape=(num_dims,))
    x = layers.Dense(num_dims, activation="tanh")(encoder_inputs)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(latent_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    decoder_outputs = layers.Dense(num_dims, activation="linear")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    class VAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                total_loss = reconstruction_loss + kl_loss_factor * kl_loss
        
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
    creditdata = np.concatenate([data], axis=0)
    creditdata = np.expand_dims(creditdata, -1).astype("float32")

    vae = VAE(encoder, decoder)
    vae.compile(optimizer=tf.keras.optimizers.Adam())
    history = vae.fit(creditdata,epochs=epochs,batch_size=batch_size,verbose=1)

    return vae, history

In [ ]:
vae, history = train_VAE(data, num_dims=784, hidden_layer_n=[512, 256, 128] )

In [ ]:
sample = data[3, :].reshape([1, 784])

z_mean, z_log_var, z = vae.encoder(sample.astype('float32'))
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample.astype('float32'),reconstruction)
        

In [ ]:
reconstruction_loss.numpy()

In [ ]:
def vae_detect_outliers(data,
                        vae_model,
                        num_dims
                        ):

    data_mean = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1, num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        data_mean.append(reconstruction_loss)
    
    data_mean = np.array(data_mean)
    i_mean = np.mean(data_mean)
    i_std = np.std(data_mean)
    

    threshold = i_mean + 3*i_std

    classes = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        if reconstruction_loss > threshold:
            
            classes.append(1)
            
        else:
            
            classes.append(0)

    classes = np.array(classes)

    return classes

In [ ]:
classes = vae_detect_outliers(data, vae, 784)

In [ ]:
np.sum(classes)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
# sklearn.metrics.confusion_matrix(lables, 1-classes, labels=None, sample_weight=None, normalize=None)
true_labels = 1- lables
predicted_labels = classes

cm = confusion_matrix(true_labels, predicted_labels)

# Normalize the confusion matrix to get percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print("Confusion Matrix (Percentage):")
print(cm_percentage)

In [ ]:
cm

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

true_labels = 1- lables
predicted_labels = classes

precision = precision_score(true_labels, predicted_labels)

# Calculate recall
recall = recall_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
mat[:, 0] = mat[:, 0]/(mat[0, 0]+ mat[1, 0])
mat[:, 1] = mat[:, 1]/(mat[0, 1]+ mat[1, 1])

In [ ]:
mat

## RAE

In [ ]:
def train_RAE(data,
              latent_dim = 2,
              hidden_layer_n = [20,18,16],
              num_dims = 10,
              z_loss_w = 0.01,
              REG_loss_w = 0.01,
              epochs = 100,
              batch_size = 128
              ):


    """
    Training the RAE on the data
    """


    encoder_inputs = keras.Input(shape=(num_dims,))
    x = layers.Dense(hidden_layer_n[0], activation="sigmoid")(encoder_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="sigmoid")(x)
    x = layers.Dense(hidden_layer_n[2], activation="sigmoid")(x)
    encoder_output = layers.Dense(latent_dim, activation="sigmoid")(x)
    encoder = keras.Model(encoder_inputs, encoder_output, name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(hidden_layer_n[2], activation="sigmoid")(latent_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="sigmoid")(x)
    x = layers.Dense(hidden_layer_n[0], activation="sigmoid")(x)
    decoder_outputs = layers.Dense(num_dims, activation="linear")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



    class RAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.z_tracker = keras.metrics.Mean(name="z_loss")
            self.REG_tracker = keras.metrics.Mean(name="REG_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.z_tracker,
                self.REG_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape(persistent=True) as tape:
                z = self.encoder(data)
                reconstruction = self.decoder(z)

                reconstruction_loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)(data,reconstruction)

                z_loss = K.mean(K.square(z), axis=[1])
        
                REG_loss = K.mean(K.square(K.gradients(K.square(reconstruction), z)))

#                 z_loss_w = z_loss_w
#                 REG_loss_w = REG_loss_w

                total_loss = reconstruction_loss +  z_loss_w * z_loss + REG_loss_w * REG_loss
            
                grads = tape.gradient(total_loss, self.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
                self.total_loss_tracker.update_state(total_loss)
                self.reconstruction_loss_tracker.update_state(reconstruction_loss)
                self.z_tracker.update_state(z_loss)
                self.REG_tracker.update_state(REG_loss)
                del tape
                return {
                    "loss": self.total_loss_tracker.result(),
                    "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                    "z_loss": self.z_tracker.result(),
                    "REG_loss": self.REG_tracker.result(),
                }

    tdata = np.concatenate([data], axis=0)
    tdata = np.expand_dims(tdata, -1).astype("float32")

    rae = RAE(encoder, decoder)
    rae.compile(optimizer=tf.keras.optimizers.Adam())
    history = rae.fit(tdata, epochs=epochs, batch_size=batch_size, verbose=1)



    return rae, history

In [ ]:
rae, history = train_RAE(data, num_dims=784, hidden_layer_n=[512, 256, 128], z_loss_w = 0.01)

In [ ]:
sample = data[1, :].reshape([1, 784])

z = rae.encoder(sample.astype('float32'))
reconstruction = rae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample.astype('float32'), reconstruction) 

In [ ]:
reconstruction_loss.numpy()

In [ ]:
def rae_detect_outliers(data,
                        rae_model,
                        num_dims
                        ):

    data_mean = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z = rae_model.encoder(sample)
        reconstruction = rae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample, reconstruction)
        
        data_mean.append(reconstruction_loss)
    
    data_mean = np.array(data_mean)
    data_std = np.std(data_mean)

    threshold = data_mean + 3*data_std

    classes = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')
        
        z = rae_model.encoder(sample)
        reconstruction = rae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample, reconstruction)
        
        if any(reconstruction_loss > threshold):
            
            classes.append(1)
            
        else:
            
            classes.append(0)

    classes = np.array(classes)


    return classes

In [ ]:
classes = rae_detect_outliers(data, rae, 784)

In [ ]:
np.sum(classes)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
# sklearn.metrics.confusion_matrix(lables, 1-classes, labels=None, sample_weight=None, normalize=None)
true_labels = 1 - lables
predicted_labels = classes

cm = confusion_matrix(true_labels, predicted_labels)

# Normalize the confusion matrix to get percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print("Confusion Matrix:")
print(cm)

print("Confusion Matrix (Percentage):")
print(cm_percentage)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

true_labels = 1- lables
predicted_labels = classes

precision = precision_score(true_labels, predicted_labels)

# Calculate recall
recall = recall_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# mnist-25

In [3]:
data, labels = csv_data_loader("mnist-25")

In [4]:
data.shape

(7090, 784)

In [5]:
labels.shape

(7090,)

## VAE

In [27]:

def train_VAE(data,
              latent_dim = 2,
              hidden_layer_n = [20,18,16],
              num_dims = 10,
              kl_loss_factor = 0.1,
              epochs = 200,
              batch_size = 64
              ):


    """
    Training the VAE on the data
    """

    class Sampling(layers.Layer):
        """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    latent_dim = latent_dim

    encoder_inputs = keras.Input(shape=(num_dims,))
    x = layers.Dense(num_dims, activation="tanh")(encoder_inputs)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(hidden_layer_n[2], activation="tanh")(latent_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="tanh")(x)
    x = layers.Dense(hidden_layer_n[0], activation="tanh")(x)
    decoder_outputs = layers.Dense(num_dims, activation="linear")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    class VAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                total_loss = reconstruction_loss + kl_loss_factor * kl_loss
        
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
    creditdata = np.concatenate([data], axis=0)
    creditdata = np.expand_dims(creditdata, -1).astype("float32")

    vae = VAE(encoder, decoder)
    vae.compile(optimizer=tf.keras.optimizers.Adam())
    history = vae.fit(creditdata,epochs=epochs,batch_size=batch_size,verbose=1)

    return vae, history

In [28]:
vae, history = train_VAE(data, num_dims=784, hidden_layer_n=[512, 256, 128] )

Epoch 1/200
111/111 [==============================] - 2s 15ms/step - loss: 7483.5681 - reconstruction_loss: 6718.9336 - kl_loss: 9.9495
Epoch 2/200
111/111 [==============================] - 2s 22ms/step - loss: 5235.6359 - reconstruction_loss: 4986.9756 - kl_loss: 2.2114
Epoch 3/200
111/111 [==============================] - 2s 16ms/step - loss: 4511.4428 - reconstruction_loss: 4432.0234 - kl_loss: 1.5608
Epoch 4/200
111/111 [==============================] - 2s 15ms/step - loss: 4287.9073 - reconstruction_loss: 4271.2686 - kl_loss: 2.6339
Epoch 5/200
111/111 [==============================] - 2s 17ms/step - loss: 4227.0034 - reconstruction_loss: 4231.8589 - kl_loss: 2.0421
Epoch 6/200
111/111 [==============================] - 2s 19ms/step - loss: 4230.7820 - reconstruction_loss: 4224.0093 - kl_loss: 1.0265
Epoch 7/200
111/111 [==============================] - 2s 19ms/step - loss: 4218.2061 - reconstruction_loss: 4223.2607 - kl_loss: 5.4165
Epoch 8/200
111/111 [====================

111/111 [==============================] - 2s 16ms/step - loss: 4234.1325 - reconstruction_loss: 4223.4985 - kl_loss: 3.3094
Epoch 61/200
111/111 [==============================] - 2s 15ms/step - loss: 4221.1360 - reconstruction_loss: 4223.1338 - kl_loss: 3.1506
Epoch 62/200
111/111 [==============================] - 2s 15ms/step - loss: 4216.4707 - reconstruction_loss: 4223.3237 - kl_loss: 2.9255
Epoch 63/200
111/111 [==============================] - 2s 14ms/step - loss: 4212.5886 - reconstruction_loss: 4223.4004 - kl_loss: 2.6802
Epoch 64/200
111/111 [==============================] - 2s 16ms/step - loss: 4235.5768 - reconstruction_loss: 4223.3296 - kl_loss: 2.2814
Epoch 65/200
111/111 [==============================] - 2s 15ms/step - loss: 4229.8571 - reconstruction_loss: 4222.5605 - kl_loss: 2.9550
Epoch 66/200
111/111 [==============================] - 2s 16ms/step - loss: 4230.0693 - reconstruction_loss: 4223.2363 - kl_loss: 3.1133
Epoch 67/200
111/111 [=========================

111/111 [==============================] - 2s 14ms/step - loss: 4238.2654 - reconstruction_loss: 4223.0391 - kl_loss: 7.6988e-07
Epoch 119/200
111/111 [==============================] - 2s 14ms/step - loss: 4230.1164 - reconstruction_loss: 4223.1675 - kl_loss: 2.5345e-07
Epoch 120/200
111/111 [==============================] - 2s 15ms/step - loss: 4197.6064 - reconstruction_loss: 4223.2778 - kl_loss: 7.3249e-08
Epoch 121/200
111/111 [==============================] - 2s 18ms/step - loss: 4217.5277 - reconstruction_loss: 4223.6519 - kl_loss: 1.8293e-08
Epoch 122/200
111/111 [==============================] - 2s 20ms/step - loss: 4232.5534 - reconstruction_loss: 4223.8164 - kl_loss: 0.0000e+00
Epoch 123/200
111/111 [==============================] - 2s 21ms/step - loss: 4239.2033 - reconstruction_loss: 4223.3271 - kl_loss: 0.0000e+00
Epoch 124/200
111/111 [==============================] - 2s 18ms/step - loss: 4205.7422 - reconstruction_loss: 4223.3750 - kl_loss: 0.0000e+00
Epoch 125/200

111/111 [==============================] - 2s 15ms/step - loss: 4232.2221 - reconstruction_loss: 4222.5933 - kl_loss: 1.3123e-08
Epoch 176/200
111/111 [==============================] - 2s 14ms/step - loss: 4247.8536 - reconstruction_loss: 4223.1841 - kl_loss: 1.1749e-08
Epoch 177/200
111/111 [==============================] - 2s 16ms/step - loss: 4203.8900 - reconstruction_loss: 4223.1250 - kl_loss: 1.0492e-08
Epoch 178/200
111/111 [==============================] - 2s 17ms/step - loss: 4218.2171 - reconstruction_loss: 4223.5034 - kl_loss: 1.7873e-08
Epoch 179/200
111/111 [==============================] - 2s 18ms/step - loss: 4229.4735 - reconstruction_loss: 4223.4570 - kl_loss: 5.5990e-09
Epoch 180/200
111/111 [==============================] - 2s 14ms/step - loss: 4235.7914 - reconstruction_loss: 4222.7100 - kl_loss: 1.6246e-08
Epoch 181/200
111/111 [==============================] - 2s 14ms/step - loss: 4210.2179 - reconstruction_loss: 4223.1904 - kl_loss: 1.6141e-08
Epoch 182/200

In [31]:
sample = data[1, :].reshape([1, 784])

z_mean, z_log_var, z = vae.encoder(sample.astype('float32'))
reconstruction = vae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample.astype('float32'),reconstruction)
        

In [32]:
reconstruction_loss.numpy()

3371.2625

In [33]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [38]:
def vae_detect_outliers(data,
                        vae_model,
                        num_dims
                        ):

    data_mean = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1, num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        data_mean.append(reconstruction_loss)
    
    data_mean = np.array(data_mean)
    i_mean = np.mean(data_mean)
    i_std = np.std(data_mean)
    

    threshold = i_mean + 2*i_std

    classes = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z_mean, z_log_var, z = vae_model.encoder(sample)
        reconstruction = vae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample,reconstruction)
        
        if reconstruction_loss > threshold:
            
            classes.append(1)
            
        else:
            
            classes.append(0)

    classes = np.array(classes)

    return classes

In [39]:
classes = vae_detect_outliers(data, vae, 784)

In [40]:
np.sum(classes)

272

In [41]:
from sklearn.metrics import confusion_matrix
import numpy as np
# sklearn.metrics.confusion_matrix(lables, 1-classes, labels=None, sample_weight=None, normalize=None)
true_labels = 1- labels
predicted_labels = classes

cm = confusion_matrix(true_labels, predicted_labels)

# Normalize the confusion matrix to get percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print("Confusion Matrix:")
print(cm)

print("Confusion Matrix (Percentage):")
print(cm_percentage)

Confusion Matrix:
[[6749  241]
 [  69   31]]
Confusion Matrix (Percentage):
[[0.96552217 0.03447783]
 [0.69       0.31      ]]


## RAE

In [42]:
def train_RAE(data,
              latent_dim = 2,
              hidden_layer_n = [20,18,16],
              num_dims = 10,
              z_loss_w = 0.01,
              REG_loss_w = 0.01,
              epochs = 100,
              batch_size = 128
              ):


    """
    Training the RAE on the data
    """


    encoder_inputs = keras.Input(shape=(num_dims,))
    x = layers.Dense(hidden_layer_n[0], activation="sigmoid")(encoder_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="sigmoid")(x)
    x = layers.Dense(hidden_layer_n[2], activation="sigmoid")(x)
    encoder_output = layers.Dense(latent_dim, activation="sigmoid")(x)
    encoder = keras.Model(encoder_inputs, encoder_output, name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(hidden_layer_n[2], activation="sigmoid")(latent_inputs)
    x = layers.Dense(hidden_layer_n[1], activation="sigmoid")(x)
    x = layers.Dense(hidden_layer_n[0], activation="sigmoid")(x)
    decoder_outputs = layers.Dense(num_dims, activation="linear")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



    class RAE(keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super().__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.z_tracker = keras.metrics.Mean(name="z_loss")
            self.REG_tracker = keras.metrics.Mean(name="REG_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.z_tracker,
                self.REG_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape(persistent=True) as tape:
                z = self.encoder(data)
                reconstruction = self.decoder(z)

                reconstruction_loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)(data,reconstruction)

                z_loss = K.mean(K.square(z), axis=[1])
        
                REG_loss = K.mean(K.square(K.gradients(K.square(reconstruction), z)))

#                 z_loss_w = z_loss_w
#                 REG_loss_w = REG_loss_w

                total_loss = reconstruction_loss +  z_loss_w * z_loss + REG_loss_w * REG_loss
            
                grads = tape.gradient(total_loss, self.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
                self.total_loss_tracker.update_state(total_loss)
                self.reconstruction_loss_tracker.update_state(reconstruction_loss)
                self.z_tracker.update_state(z_loss)
                self.REG_tracker.update_state(REG_loss)
                del tape
                return {
                    "loss": self.total_loss_tracker.result(),
                    "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                    "z_loss": self.z_tracker.result(),
                    "REG_loss": self.REG_tracker.result(),
                }

    tdata = np.concatenate([data], axis=0)
    tdata = np.expand_dims(tdata, -1).astype("float32")

    rae = RAE(encoder, decoder)
    rae.compile(optimizer=tf.keras.optimizers.Adam())
    history = rae.fit(tdata, epochs=epochs, batch_size=batch_size, verbose=1)



    return rae, history

In [43]:
rae, history = train_RAE(data, num_dims=784, hidden_layer_n=[512, 256, 128], z_loss_w = 0.01)

Epoch 1/100


56/56 [==============================] - 1s 15ms/step - loss: 7912.1681 - reconstruction_loss: 7419.7925 - z_loss: 0.4465 - REG_loss: 0.6412
Epoch 2/100
56/56 [==============================] - 1s 17ms/step - loss: 6213.7749 - reconstruction_loss: 5947.7300 - z_loss: 0.3995 - REG_loss: 0.0272
Epoch 3/100
56/56 [==============================] - 1s 17ms/step - loss: 5298.3408 - reconstruction_loss: 5155.2500 - z_loss: 0.0089 - REG_loss: 7.8722e-04
Epoch 4/100
56/56 [==============================] - 1s 20ms/step - loss: 4785.3583 - reconstruction_loss: 4713.6729 - z_loss: 0.0012 - REG_loss: 5.4963e-06
Epoch 5/100
56/56 [==============================] - 1s 19ms/step - loss: 4501.2166 - reconstruction_loss: 4472.1777 - z_loss: 7.5252e-04 - REG_loss: 7.8590e-07
Epoch 6/100
56/56 [==============================] - 1s 17ms/step - loss: 4364.1955 - reconstruction_loss: 4344.2734 - z_loss: 5.2294e-04 - REG_loss: 2.2202e-07
Epoch 7/100
56/56 [==============================] - 1s 17ms/step - lo

56/56 [==============================] - 1s 17ms/step - loss: 4234.2116 - reconstruction_loss: 4222.3110 - z_loss: 1.6170e-05 - REG_loss: 2.6557e-10
Epoch 52/100
56/56 [==============================] - 1s 16ms/step - loss: 4207.4328 - reconstruction_loss: 4222.4355 - z_loss: 1.5634e-05 - REG_loss: 2.1744e-10
Epoch 53/100
56/56 [==============================] - 1s 16ms/step - loss: 4224.6712 - reconstruction_loss: 4222.5254 - z_loss: 1.5119e-05 - REG_loss: 9.7779e-10
Epoch 54/100
56/56 [==============================] - 1s 17ms/step - loss: 4216.4795 - reconstruction_loss: 4222.6304 - z_loss: 1.4623e-05 - REG_loss: 3.3339e-10
Epoch 55/100
56/56 [==============================] - 1s 19ms/step - loss: 4215.0980 - reconstruction_loss: 4222.4893 - z_loss: 1.4147e-05 - REG_loss: 3.6504e-10
Epoch 56/100
56/56 [==============================] - 1s 20ms/step - loss: 4209.8094 - reconstruction_loss: 4222.6201 - z_loss: 1.3689e-05 - REG_loss: 5.4081e-10
Epoch 57/100
56/56 [=====================

In [46]:
sample = data[-1, :].reshape([1, 784])

z = rae.encoder(sample.astype('float32'))
reconstruction = rae.decoder(z)

reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample.astype('float32'), reconstruction) 

In [47]:
reconstruction_loss.numpy()

6912.768

In [48]:
def rae_detect_outliers(data,
                        rae_model,
                        num_dims
                        ):

    data_mean = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')

        z = rae_model.encoder(sample)
        reconstruction = rae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample, reconstruction)
        
        data_mean.append(reconstruction_loss)
    
    data_mean = np.array(data_mean)
    data_std = np.std(data_mean)

    threshold = data_mean + 3*data_std

    classes = []

    for i in range(data.shape[0]):
            
        sample = data[i,:].reshape([1,num_dims])
        sample = sample.astype('float32')
        
        z = rae_model.encoder(sample)
        reconstruction = rae_model.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(sample, reconstruction)
        
        if any(reconstruction_loss > threshold):
            
            classes.append(1)
            
        else:
            
            classes.append(0)

    classes = np.array(classes)


    return classes

In [ ]:
classes = rae_detect_outliers(data, rae, 784)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
# sklearn.metrics.confusion_matrix(lables, 1-classes, labels=None, sample_weight=None, normalize=None)
true_labels = 1 - lables
predicted_labels = classes

cm = confusion_matrix(true_labels, predicted_labels)

# Normalize the confusion matrix to get percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print("Confusion Matrix:")
print(cm)

print("Confusion Matrix (Percentage):")
print(cm_percentage)